In [1]:
from color_code import color_code
import numpy as np
import copy
from timer import timer
import itertools as itt
import statsmodels.stats.proportion as prop
from joblib import Parallel, delayed

In [2]:
class batch_params:
    def __init__ (self, 
                  shots, 
                  error_rate, 
                  c,
                  rG2,
                  template_G,
                  template_rG,
                  template_rG2,
                  loop_logical,
                  string_logical,
                  boundary_logical,
                  rel_weights):
        self.shots = shots
        self.error_rate = error_rate
        self.c = c
        self.rG2 = rG2

        self.template_G = template_G
        self.template_rG = template_rG
        self.template_rG2 = template_rG2

        self.loop_logical = loop_logical
        self.string_logical = string_logical
        self.boundary_logical = boundary_logical
        self.rel_weights = rel_weights


In [3]:
def run_batch (params, n):
    shots = params.shots
    error_rate = params.error_rate
    c = params.c
    rG2 = params.rG2

    template_G = params.template_G
    template_rG = params.template_rG
    template_rG2 = params.template_rG2

    loop_logical = params.loop_logical
    string_logical = params.string_logical
    boundary_logical = params.boundary_logical

    rel_weights = params.rel_weights

    matching_info = c.make_matching(0.1,rel_weights=rel_weights,auto_boundary=False)

    logical_dict = dict()
    for logical_class in itt.product([0,1],[0,1],[0,1]):
        logical_dict[logical_class] = 0

    for i in range(0,shots):
        c.make_errors(error_rate)
    
        c.mark_errors()

        errors = c.extract_error_set()
        
        
        correction = c.move_errors_off_wall() 
        
        c.mark_errors()

        c.mark_errors_restricted_lattice()    

        decoding = color_code.decode_restricted_single(c.rG,*matching_info)
        c.mark_restricted_edges(c.rG,decoding)
        
        c.mark_modified_restricted_lattice(rG2,c.rG)

        decoding_errors = c.lift_restricted_lattice(rG2)
        
        
        decoding = [node for node in decoding_errors.keys() if decoding_errors[node]]
        decoding = c.combine_errors(decoding,correction)
        
        assert c.check_logical(errors,decoding,G=template_G)

        logical_result = tuple(c.identify_logical(c.combine_errors(errors,decoding),[boundary_logical,string_logical,loop_logical]))

        logical_dict[logical_result] += 1

        c.wipe_errors()
        c.unmark_restricted_edges(c.rG)
        c.unmark_restricted_edges(rG2)

    return logical_dict


In [4]:
logical_dicts = list()
tmr = timer()

p_range = list(np.linspace(0.08,0.10,20))
d_range = [15]
for d in d_range:
    for p in p_range:
        c = color_code(int(d),0.01)  
        twist = ((1,-6,5),(1,6,-7))
        pairs1, pairs2, shared_checks = c.insert_twist(*twist)

        c.generate_combined_restricted_lattice("red").nodes(data=True)
        rG2 = c.modify_combined_restricted_lattice(c.rG)
        c.mark_modified_restricted_lattice_prepoc(c.rG)
        c.move_errors_off_wall_preproc()
        c.lift_restricted_lattice_prepoc(rG2)

        loop_logical = c.make_twist_loop_logicals(twist)
        boundary_logical = c.make_boundary_logical("red")
        string_logical = c.make_crossing_logical(twist)

        template_G = copy.deepcopy(c.G)
        template_rG = copy.deepcopy(c.rG)
        template_rG2 = copy.deepcopy(rG2)

        shots = 10000000
        batch_shots = 2000
        logical_dict = dict()
        error_rate = p

        rel_weights = (2.0,2.0,2.0)
        
        params = batch_params(batch_shots,
                            error_rate,
                            c,
                            rG2,
                            template_G,
                            template_rG,
                            template_rG2,
                            loop_logical,
                            string_logical,
                            boundary_logical,
                            rel_weights)
        
        n_jobs = 12
        correct_decodings = 0
        current_shots = 0

        for i in range(0,int(shots/(batch_shots*n_jobs))):

            tmr.start()
            batch_results = Parallel(n_jobs=-1,verbose=10,backend='loky')(delayed(run_batch)(params,i) for i in range(n_jobs))
            tmr.stop()

            current_shots += (n_jobs * batch_shots)
        
            for logical_class in itt.product([0,1],[0,1],[0,1]):
                if not (logical_class in logical_dict.keys()):
                    logical_dict[logical_class] = 0
                logical_dict[logical_class] += np.sum([result[logical_class] for result in batch_results])

            correct_decodings = logical_dict[(0,0,0)]

            print("ERROR RATE", p)
            print("TIME FOR BATCH,",tmr.get_time())
            tmr.reset()
            print("PARAMS",rel_weights)
            print("COMPLETION RATE",str(current_shots)+"/"+str(shots))

            success_rate = correct_decodings/current_shots
            print("PERCENTAGE CORRECT,",success_rate)
            confint = prop.proportion_confint(correct_decodings,current_shots,method='binom_test')
            print("CONFIDENCE",confint)
            print("CONFIDENCE %",100*(confint[1]-confint[0])/(1-success_rate))
            print("-----------------------------------------------")
            if confint[1]-confint[0] < (1-success_rate)*0.05 and current_shots > 20000:
                break
            
                
        logical_dicts.append(logical_dict)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   25.9s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   26.3s remaining:  1.3min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   28.6s remaining:   40.1s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   29.4s remaining:   20.9s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   30.1s remaining:   10.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   30.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


ERROR RATE 0.08
TIME FOR BATCH, 30.762860536575317
PARAMS (2.0, 2.0, 2.0)
COMPLETION RATE 24000/10000000
PERCENTAGE CORRECT, 0.613625
CONFIDENCE (0.6074387731544615, 0.619772241973751)
CONFIDENCE % 3.1920980444618667
-----------------------------------------------


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   23.4s remaining:  1.2min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   23.8s remaining:   33.3s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   24.2s remaining:   17.3s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   24.9s remaining:    8.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   26.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


ERROR RATE 0.08
TIME FOR BATCH, 26.134564638137817
PARAMS (2.0, 2.0, 2.0)
COMPLETION RATE 48000/10000000
PERCENTAGE CORRECT, 0.6143125
CONFIDENCE (0.6099485605332264, 0.6186673732589854)
CONFIDENCE % 2.2605899143111765
-----------------------------------------------


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   22.5s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   23.2s remaining:   32.5s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   23.7s remaining:   16.9s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   24.4s remaining:    8.1s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   24.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   20.9s remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   21.0s remaining:   29.4s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   21.1s remaining:   15.0s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   21.6s remaining:    7.1s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   21.8s finished


ERROR RATE 0.1
TIME FOR BATCH, 21.9492826461792
PARAMS (2.0, 2.0, 2.0)
COMPLETION RATE 360000/10000000
PERCENTAGE CORRECT, 0.46336666666666665
CONFIDENCE (0.46173747005705873, 0.4649971947822501)
CONFIDENCE % 0.6074398518898139
-----------------------------------------------


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   22.3s remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   22.5s remaining:   31.5s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   22.6s remaining:   16.1s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   22.7s remaining:    7.5s


ERROR RATE 0.1
TIME FOR BATCH, 22.962197065353394
PARAMS (2.0, 2.0, 2.0)
COMPLETION RATE 384000/10000000
PERCENTAGE CORRECT, 0.4632526041666667
CONFIDENCE (0.4616757532303588, 0.4648307034099533)
CONFIDENCE % 0.5877904958805141
-----------------------------------------------


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   22.9s finished


In [5]:
for dict in logical_dicts:
    total = 0
    for key in dict.keys():
        total += dict[key]
    for key in dict.keys():
        dict[key] = dict[key]/total

In [6]:
logical_dicts

[{(0, 0, 0): 0.6157543103448276,
  (0, 0, 1): 0.13964511494252874,
  (0, 1, 0): 0.009172413793103448,
  (0, 1, 1): 0.010801724137931035,
  (1, 0, 0): 0.03824425287356322,
  (1, 0, 1): 0.08402442528735632,
  (1, 1, 0): 0.05172988505747127,
  (1, 1, 1): 0.05062787356321839},
 {(0, 0, 0): 0.606922619047619,
  (0, 0, 1): 0.1421577380952381,
  (0, 1, 0): 0.009886904761904762,
  (0, 1, 1): 0.011322916666666667,
  (1, 0, 0): 0.040223214285714286,
  (1, 0, 1): 0.0846547619047619,
  (1, 1, 0): 0.05297767857142857,
  (1, 1, 1): 0.051854166666666666},
 {(0, 0, 0): 0.5984691358024692,
  (0, 0, 1): 0.1456141975308642,
  (0, 1, 0): 0.01016358024691358,
  (0, 1, 1): 0.012003086419753086,
  (1, 0, 0): 0.040726851851851854,
  (1, 0, 1): 0.08489197530864198,
  (1, 1, 0): 0.05481172839506173,
  (1, 1, 1): 0.05331944444444445},
 {(0, 0, 0): 0.589801282051282,
  (0, 0, 1): 0.14725320512820514,
  (0, 1, 0): 0.010740384615384615,
  (0, 1, 1): 0.012576923076923078,
  (1, 0, 0): 0.042479166666666665,
  (1, 0, 